In [1]:
import os
import cv2 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
def get_essential(img):
    row=[]
    column=[]
    for i in range(267):
        temprow=[]
        tempcolumn=[]
        for j in range(267):
            temprow.append(img[i][j])
            tempcolumn.append(img[j][i])
        row.append(np.mean(temprow))
        column.append(np.mean(tempcolumn))

    return [row,column]

In [3]:
rightLabels = ['1', '3', '5', '7']
labelsTrain = ['0', '1', '2', '3']
def get_data(data_dir):
    data = [] 
    l=[]
    for label in labelsTrain:
        lbl=rightLabels[labelsTrain.index(label)]
        path = os.path.join(data_dir, lbl)
        subs= next(os.walk(path))[1]
        for sub in subs:
            imgs=next(os.walk(path+"/"+sub))[2]
            temp=[]
            lenght= len(imgs)
            resto= 39-lenght
            if resto==0:
                parte1=0
                parte2=0
            elif (resto%2)==0:
                parte1=resto/2
                parte2=resto/2
            else:
                parte1=(resto/2)+0.5
                parte2=(resto/2)-0.5
            for n in range(int(parte1)):
                temp.append(np.zeros((2,267)))
            for img in imgs:
                img_arr = cv2.imread(os.path.join(path+"/"+sub, img), 0)
                img_arr= cv2.resize(img_arr, (267,267))
                threshold =0.2
                img_arr= np.where(img_arr/255>=threshold, 1, 0)
                temp.append(get_essential(img_arr))
            for n in range(int(parte2)):
                temp.append(np.zeros((2,267)))
            l.append(label)
            data.append(np.asarray(temp))
    return np.asarray(data), to_categorical(np.asarray(l).astype(np.float32))

In [4]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=False):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
	# define model
	print(trainX.shape)
	verbose, epochs, batch_size = 1, 100, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape
	n_length = 267
	trainX = trainX.reshape((trainX.shape[0], n_timesteps, n_features, n_length, 1))
	testX = testX.reshape(testX.shape[0], n_timesteps, n_features, n_length, 1)
	# define model
	model = Sequential()
	model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_timesteps, n_features, n_length, 1)))
	model.add(Dropout(0.3))
	model.add(Flatten())
	model.add(Dense(256, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	history= model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data=(testX, testy))
	model.summary()

	# evaluate model
	y_pred = model.predict(testX, batch_size)
	y_pred= np.argmax(y_pred, axis=1)
	y_label= np.argmax(testy, axis=1)
	print('Classification Report')
	print(classification_report(y_label, y_pred, target_names=rightLabels))
	print('Confusion Matrix')
	plot_confusion_matrix(confusion_matrix(y_label, y_pred), rightLabels)
	
	print("finish")
	return history
 
# summarize scores
def summarize_results(history):
	plt.plot(history.history['accuracy'])
	plt.plot(history.history['val_accuracy'])
	plt.title('model accuracy')
	plt.ylabel('accuracy')
	plt.xlabel('epoch')
	plt.legend(['train', 'test'], loc='upper left')
	plt.show()
	# summarize history for loss
	plt.plot(history.history['loss'])
	plt.plot(history.history['val_loss'])
	plt.title('model loss')
	plt.ylabel('loss')
	plt.xlabel('epoch')
	plt.legend(['train', 'test'], loc='upper left')
	plt.show()

 
# run an experiment
def run_experiment(repeats=10):
	# load data
	x, y = get_data('train_dataset/')
	# repeat experiment
	for r in range(repeats):
		trainx, testx, trainy, testy = train_test_split(x, y, test_size=0.30, random_state=1, shuffle=True)
		print(trainx.shape, trainy.shape, testx.shape,testy.shape)
		history = evaluate_model(trainx, trainy, testx, testy)
		# summarize results
		summarize_results(history)
 
# run the experiment

run_experiment()

KeyboardInterrupt: 